# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [14]:
import pandas as pd
import numpy as np
from scipy import stats

In [15]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [20]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [16]:
data.head()


,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [13]:
#check all the columns in the dataframe
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')


1. What test is appropriate for this problem? Does CLT apply?

A. This is a binomial distribution problem. Therefore CLT does not apply directly. 
However, the difference in % call back between the races will be normally distributed since the sample size is large enough. 
A two tail t- test is appropriate to test the probablity of call back between the races.



2. What are the null and alternate hypotheses?

A. Null Hypothesis (H0): Race has no bearing on the interview calls. Meaning, race(w) = race(b) for call =1
   Alternate hpothesis(Ha): Race has effect on the number of call backs. race(w) != race(b) for call =1

In [34]:
# number of call back for BLACK sounding names
b_call = sum(data[data.race=='b'].call)
print('call back for black names:', b_call)
total_b = data[data.race=='b'].call.count()
print('total black sounding names:', total_b)
percent_b_callback = (b_call/ total_b)*100
print('percent call back for black names:',percent_b_callback)

call back for black names: 157.0
total black sounding names: 2435
percent call back for black names: 6.447638603696099


In [35]:
# number of call back for WHITE sounding names
w_call = sum(data[data.race=='w'].call)
print('call back for white names:', w_call)
total_w = data[data.race=='w'].call.count()
print('total white sounding names:', total_w)
percent_w_callback = (w_call/ total_w)*100
print('percent call back for black names:',percent_w_callback)

call back for white names: 235.0
total white sounding names: 2435
percent call back for black names: 9.650924024640657


In [44]:
# percentage difference in call back
percent_diff = abs(percent_b_callback - percent_w_callback)
print('percent diff in call back:', percent_diff)

percent diff in call back: 3.2032854209445585


In [37]:
# percentage callback Black variance
P1=percent_b_callback/100
n1=total_b
var_b=(P1*(1-P1)/n1)
var_b

2.4771737856498466e-05

In [38]:
# percentage callback white variance
P2=percent_w_callback/100
n2=total_w
var_w=(P2*(1-P2)/n2)
var_w

3.580911983304638e-05

In [42]:
# Sampling Distribution P1-P2 variance and standard deviation
var_b_w= var_b + var_w
print('Sampling distribution P1-P2 variance:',var_b_w)
std_b_w = np.sqrt(var_b_w)
print('Sampling distribution P1-P2 std:',std_b_w)

Sampling distribution P1-P2 variance: 6.058085768954485e-05
Sampling distribution P1-P2 std: 0.0077833705866767544


3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

Margin of error (moe), confidence interval and p-values are calculated below.

In [74]:
# Q3: Compute margin of error, confidence interval, and p-value.
# margin of error at 95% CI
moe = 1.96*std_b_w
print('margin of error:', moe)

# confidence interval
ci = abs(P1-P2) + np.array([-1, 1]) * moe
print('CI:',ci)

# Standard Error
print('Standard Error:', std_b_w)
#p-value
n =2435 # total samples
tt = ((P1-P2)-0)/std_b_w
p_value = stats.t.sf(np.abs(tt), 2*n-1)*2 # two-sided pvalue = Prob(abs(t)>tt)
print('t statistic:', tt)
print('p_value:',p_value)

margin of error: 0.015255406349886438
CI: [0.01677745 0.04728826]
Standard Error: 0.0077833705866767544
t statistic: -4.11555043573
p_value: 3.926425924210384e-05


In [77]:
# P-value inddepndent t-test
b_call = data[data.race=='b'].call
w_call = data[data.race=='w'].call
stats.ttest_ind(b_call,w_call,equal_var=False)

Ttest_indResult(statistic=-4.114705290861751, pvalue=3.942941513645935e-05)


4. Write a story describing the statistical significance in the context or the original problem.

A. The probablity of 3.94e-05 < alpha = 0.05 and therefore the null hypothesis of percentage call back for black names = white names is not true. We can conclude that there is a difference in response rates for black sounding and white sounding names.  

5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

A. From the analysis of data for call back and race columns, we can conclude there is a racial bias. But, in order to conclude it is the most important factor in callback success, other data also needs to be analyzed as well. For example, 
there is no information about the candidate location such as west, midwest, south, far east etc. Therefore, the data could be biased/ un-biased based on region as well and does not represent the true population and can skew the conclusion. 
